In [6]:
from surprise import SVD,NMF
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, GridSearchCV,train_test_split
import pandas as pd 
from math import *

In [2]:
def Split(df,column):
    user_id = []
    movie_id = []
    for i in range(0,len(df)):
        res = df[column].iloc[i].split('_')
        user_id += [res[0]]
        movie_id += [res[1]]
    return movie_id,user_id

In [3]:
# Load the data
df_temp = pd.read_csv(r"/Users/bedanian/Downloads/d9a664be-2cd2-4ffc-b861-c772a8f5b9f6_epfml-rec-sys/data_train.csv")
(movie_id,user_id) = Split(df_temp,'Id')
dic = {'movieID':movie_id,'userID':user_id,'rating':df_temp['Prediction']}
df = pd.DataFrame(dic)

In [4]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['movieID','userID','rating']], reader)

In [12]:
param_grid = {'n_factors': [5, 20], 'n_epochs': [0, 40],'lr_all':[0.4, 0.6],'reg_all': [0.4, 0.6]}
gs = GridSearchCV(NMF, param_grid, measures = ['rmse', 'mae'], cv = 3)
gs.fit(data)
# best RMSE score
print(gs.best_score['rmse'])
# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

TypeError: __init__() got an unexpected keyword argument 'lr_all'

In [ ]:
# Load testset
df_test = pd.read_csv(r"/Users/bedanian/Downloads/d9a664be-2cd2-4ffc-b861-c772a8f5b9f6_epfml-rec-sys/sampleSubmission.csv")
(movie_id_test,user_id_test) = Split(df_test,'Id')

In [ ]:
def entiere(n):
    if n - floor(n)>= 0.5 :
        return(int(n)+1)
    else:
        return(int(n))

In [ ]:
predictions = []
for k in range(len(df_test)):
    predictions += [entiere(algo.predict(user_id_test[k],movie_id_test[k]).est)]

In [ ]:
res = pd.DataFrame({'Id':df_test['Id'],'Prediction':predictions})
res.to_csv(r"/Users/bedanian/Downloads/d9a664be-2cd2-4ffc-b861-c772a8f5b9f6_epfml-rec-sys/Submission.csv")